# Prepare and understand data for modeling
- https://12bme.tistory.com/309?category=737765


- 모든 데이터는 보통 지젖분하고 데이터가 의도한것에 대한 충분한 신뢰성을 가지고 있지 않다

- 데이터는 중복데이터나 관찰되지 않은값, 아웃라이어, 존재하지 않는 주소, 잘못된 전화번호 또는 지역코드, 지역좌표, 공백관련 문제를 가지고 잇다
- 데이터 분석가는 통계모델 또는 머신러닝 모델을 빌드하기 위해 이러한 데이터를 깨끗하게 만들어야 한다.
- 데이터는 앞에서 말한 문제점들이 없을경우 기술적으로 깨끗하다고 말할 수 잇다. 그러나 모델링목적으로 데이터셋을 깨끗하게 하기 위해서는 피처의 분포를 확인해야 하고 사전에 정의된 조건들을 만족하는지 검증해야 한다.
#### **데이터 분석가는 80 ~ 90 % 의 시간을 데이터를 다루거나 피처에 익숙해 지는데 쓰게 된다**

In [2]:
import findspark, pyspark
findspark.find()

'C:\\Bigdata\\spark-2.4.5-bin-hadoop2.7'

In [3]:
base_path = "../../data/RDD_example/"
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

conf  = pyspark.SparkConf().setAppName('appName').setMaster('local[2]')
sc    = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

## Duplicates

In [4]:
df = spark.createDataFrame([
                            (1, 144.5, 5.9, 33, 'M'),
                            (2, 167.2, 5.4, 45, 'M'),
                            (3, 124.1, 5.4, 23, 'F'),
                            (4, 144.5, 5.9, 33, 'M'),
                            (5, 133.2, 5.7, 54, 'F'),
                            (3, 124.1, 5.4, 23, 'F'),
                            (5, 129.2, 5.3, 42, 'M')
                            ], ['id', 'weight', 'height', 'age', 'gender'])

In [5]:
print( 'count of rows:{0}'.format(df.count()) )

count of rows:7


In [6]:
print( 'Count of Distinct rows: {0}'.format(df.distinct().count()) )

Count of Distinct rows: 6


In [7]:
df = df.dropDuplicates()
df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  5| 133.2|   5.7| 54|     F|
|  5| 129.2|   5.3| 42|     M|
|  1| 144.5|   5.9| 33|     M|
|  4| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.4| 23|     F|
+---+------+------+---+------+



In [8]:
print( 'count of rows:{0}'.format(df.count()) )

count of rows:6


In [9]:
# 특정컬럼을 제외한 리스트
cols = [c for c in df.columns if c != 'id']
cols

['weight', 'height', 'age', 'gender']

In [10]:
print( 'Count of Distinct ids: {0}'.format(df.select(cols).distinct().count()) )

Count of Distinct ids: 5


In [11]:
df = df.dropDuplicates(subset=[c for c in df.columns if c != 'id'])
df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  5| 133.2|   5.7| 54|     F|
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.4| 23|     F|
|  5| 129.2|   5.3| 42|     M|
+---+------+------+---+------+



In [12]:
import pyspark.sql.functions as fn

In [13]:
df.agg( fn.count('id').alias('count')
      , fn.countDistinct('id').alias('distinct') ).show()

+-----+--------+
|count|distinct|
+-----+--------+
|    5|       4|
+-----+--------+



- monotonically_increasing_id()
    - 각 행에 고유한 값 부여
        - 각각의 파티션에 800억개의 데이터가 있고 파티션의 수가 약 10억개 미만인 데이터들에 대해서는 monotonically_increasing_id()함수가 고유한 ID값을 부여해준다
        
    - 주의 : 스파크 이전버전에서 monotonically_increasing_id()ㅎ마수는 같은 데이터프레임에서 여러번 작업이 이뤄졌을때마다 ID값이 바뀌었는데 스파크 2.x 에서는 ID 고정

In [14]:
df.withColumn( 'new_id', fn.monotonically_increasing_id() ).show()

+---+------+------+---+------+-------------+
| id|weight|height|age|gender|       new_id|
+---+------+------+---+------+-------------+
|  5| 133.2|   5.7| 54|     F|  25769803776|
|  1| 144.5|   5.9| 33|     M| 171798691840|
|  2| 167.2|   5.4| 45|     M| 592705486848|
|  3| 124.1|   5.4| 23|     F| 687194767360|
|  5| 129.2|   5.3| 42|     M|1365799600128|
+---+------+------+---+------+-------------+



## Missing observations

- 미관찰 값들을 다룰 수 있는 가장 간단한 방법은 미관찰 값을 가지고 있는 모든 데이터를 제거하는것이다.

- 단, 너무 많은 데이터를 제거하지 않도록 조심해야함.
- 데이터셋 전체에서 미관찰 값의 분포에 따라 데이터셋 전체의 사용가능성에 큰 영향을 미칠 수 있다.
- 데이터를 제거한 후 아주 작은 데이터만 남거나, 데이터가 절반이상으로 줄어들었다면 어떤 피처가 빈칸을 가장 많이 가지고 있는지 확인하고 그 피쳐를 제거하는것이 더 좋다.

- 미관찰 값을 다루는 또 다른 방법은 None이라는 값으로 채우는 것이다
- 이럴경우 데이터필드 타입에 따란 몇몇 다양한 값으로 대체해 채워 넣을 수 잇다.
- 데이터가 참/ 거짓으로 구분이 되면 Missing이라는 세번째 카테고리를 넣으면 된다.
- 데이터가 이미 카테고리를 가지고 있다면 Missing 카테고리를 집어 넣으면 된다.
- 순서 혹은 숫자 데이터를 가지고 이을경우에는 평균, 중간값 또는 미리 정의된 다른값으로 바꿀 수 있다(ex 데이터 분포에 따라 첫번째, 세번째 quantile)

In [15]:
df_miss = spark.createDataFrame([ (1, 144.5, 5.6, 28, 'M', 100000)
                                 , (2, 167.2, 5.4, 45, 'M', None)
                                 , (3, None , 5.2, None, None, None)
                                 , (4, 144.5, 5.9, 33, 'M', None)
                                 , (5, 133.2, 5.7, 54, 'F', None)
                                 , (3, 124.1, 5.2, None, 'F', None)
                                 , (5, 129.2, 5.3, 42, 'M', 76000) ]
                , ['id', 'weight', 'height', 'age', 'gender', 'income'])


In [16]:
df_miss.rdd.map(
    lambda row: (row['id'], sum([c == None for c in row]))
).collect()

[(1, 0), (2, 1), (3, 4), (4, 1), (5, 1), (3, 2), (5, 0)]

## 결측치 많은 컬럼 확인

In [17]:
df_miss.where('id==3').show()

+---+------+------+----+------+------+
| id|weight|height| age|gender|income|
+---+------+------+----+------+------+
|  3|  null|   5.2|null|  null|  null|
|  3| 124.1|   5.2|null|     F|  null|
+---+------+------+----+------+------+



In [18]:
df_miss.agg(*[(1 - (fn.count(c) / fn.count('*'))).alias(c + 'missing')
             for c in df_miss.columns
             ]).show()

+---------+------------------+-------------+------------------+------------------+------------------+
|idmissing|     weightmissing|heightmissing|        agemissing|     gendermissing|     incomemissing|
+---------+------------------+-------------+------------------+------------------+------------------+
|      0.0|0.1428571428571429|          0.0|0.2857142857142857|0.1428571428571429|0.7142857142857143|
+---------+------------------+-------------+------------------+------------------+------------------+




- count() 함수에 있는 * 아규먼트(argument)를 칼럼 이름 부분에 추가하면 모든 행의 개수를 셀 수 있게 됩니다. 반면에 리스트 앞에 * 가 오면, agg() 함수는 그 리스트의 각 엘리먼트를 함수에 전달할 파라미터로 취급합니다. 


## 결측치 많은 컬럼 삭제

In [19]:
[c for c in df_miss.columns if c != 'income']

['id', 'weight', 'height', 'age', 'gender']

In [20]:
df_miss_no_income = df_miss.select([c for c in df_miss.columns if c != 'income'])

In [21]:
df_miss_no_income.columns

['id', 'weight', 'height', 'age', 'gender']

In [22]:
df_miss.select([c for c in df_miss.columns if c != 'income']).show()

+---+------+------+----+------+
| id|weight|height| age|gender|
+---+------+------+----+------+
|  1| 144.5|   5.6|  28|     M|
|  2| 167.2|   5.4|  45|     M|
|  3|  null|   5.2|null|  null|
|  4| 144.5|   5.9|  33|     M|
|  5| 133.2|   5.7|  54|     F|
|  3| 124.1|   5.2|null|     F|
|  5| 129.2|   5.3|  42|     M|
+---+------+------+----+------+



## 컬측치 많은 행 삭제

- 행에서 제거할 수 있는 최소의 미관찰 값 개수를 임계치로 설정할 수 있습니다. 이 값은 데이터셋이 수십 혹은 수백개의 피처들을 가지고 있거나 미관찰 값에 대한 임계치를 넘은 행들을 제거하고 싶을 경우 유용하게 사용할 수 있습니다.
    - 다음은 미관찰값 3개

In [23]:
df_miss_no_income.dropna(thresh=3).show()

+---+------+------+----+------+
| id|weight|height| age|gender|
+---+------+------+----+------+
|  1| 144.5|   5.6|  28|     M|
|  2| 167.2|   5.4|  45|     M|
|  4| 144.5|   5.9|  33|     M|
|  5| 133.2|   5.7|  54|     F|
|  3| 124.1|   5.2|null|     F|
|  5| 129.2|   5.3|  42|     M|
+---+------+------+----+------+



- 미관찰 값을 추정해 채우려면 filna() 함수를 사용 할 수 있다.
    - 이 함수는 단일 integer, float, long, string 타입을 지원한다.
    - 전체 데이터셋에서 미관찰값은 그 값으로 채워질 것이다.

- 평균, 중간값 또는 다른 계산된 값으로 채우려면 우선 그 값을 계산하고
    - 그러한 값을 가지는 딕셔너리를 만든 후 fillna() 함수에 전달 하면 된다.


In [24]:
means = df_miss_no_income.agg(
*[fn.mean(c).alias(c)
 for c in df_miss_no_income.columns if c != 'gender']
).toPandas().to_dict('records')[0]

- 여기서 데이터 타입이 두번 변환됩니다. 
    - agg() 함수의 결과를 취해 pandas의 데이터프레임으로 변환한 후 딕셔너리 형태로 한번 더 변환했습니다.
 
- toPandas() 함수는 RDD에서 collect() 함수와 같은 방식으로 동작하기 때문에 문제를 일으킬 수 있다는 사실을 염두해두어야 합니다. 
    - 이 함수는 모든 정보를 워커 노드로부터 수집한 후, 드라이버 노드로 옮깁니다. 
    - 이는 수천 개의 피처를 다루지 않는 한, 앞에서 다뤘던 데이터셋에서는 문제가 되지 않습니다.




In [25]:
means['gender'] = 'missing'
means

{'id': 3.2857142857142856,
 'weight': 140.45000000000002,
 'height': 5.471428571428571,
 'age': 40.4,
 'gender': 'missing'}

In [26]:
df_miss_no_income.fillna(means).show()

+---+------------------+------+---+-------+
| id|            weight|height|age| gender|
+---+------------------+------+---+-------+
|  1|             144.5|   5.6| 28|      M|
|  2|             167.2|   5.4| 45|      M|
|  3|140.45000000000002|   5.2| 40|missing|
|  4|             144.5|   5.9| 33|      M|
|  5|             133.2|   5.7| 54|      F|
|  3|             124.1|   5.2| 40|      F|
|  5|             129.2|   5.3| 42|      M|
+---+------------------+------+---+-------+



In [27]:
df_outliers = spark.createDataFrame(
                                    [ (1, 143.5, 5.3, 28)
                                    , (2, 154.2, 5.5, 45)
                                    , (3, 342.3, 5.1, 99)
                                    , (4, 144.5, 5.5, 33)
                                    , (5, 133.2, 5.4, 54)
                                    , (6, 124.1, 5.1, 21)
                                    , (7, 129.2, 5.3, 42) ]
                                  , ['id', 'weight', 'height', 'age']
)

## Outlier

- 일반적으로 대부분의 데이터와는 매우 다른 분포를 띄는 데이터를 말함
- Q1 - 1.5 IOR 또는 Q3 + 1.5 IQR 사이에 있으면 아웃라이어가 없다고 말할 수 있다.
    - IQR = 상위 쿼타일 75%와 하위 25% 쿼타일의 차로 정의

- 상/하위 제한선을 각각의 피처에 대해 계산
    - approxQuantile() 을 이용
    - 함수 인자의 설명은 다음과 같다
        1. 컬럼명
        1. 0~1 사이의 값 ( 0.5 중간값 )
        1. 각 피처에 대한 허용 가능한 수준의 에러
            - 0 으로 지정시 피처에 대한 정확한 값을 계산 할 수 있으나 매우 많은 연산량요구

In [28]:
cols = ['weight', 'height', 'age']
bounds = {}

for col in cols:
    quantiles = df_outliers.approxQuantile( col, [0.25, 0.75], 0.05 )
    IQR = quantiles[1] - quantiles[0]
    bounds[col] = [ quantiles[0] - 1.5 * IQR, quantiles[1] + 1.5 * IQR ]

In [41]:
# 이제 아웃라이어를 확인하기 위해 이 값을 사용해 보겠습니다.
outliers = df_outliers.select(
    *['id'] + [( (df_outliers[c] < bounds[c][0])
            |(df_outliers[c] > bounds[c][1])
           ).alias(c + '_o') for c in cols]
)

outliers.show()

+---+--------+--------+-----+
| id|weight_o|height_o|age_o|
+---+--------+--------+-----+
|  7|   false|   false|false|
|  6|   false|   false|false|
|  5|   false|   false|false|
|  1|   false|   false|false|
|  3|    true|   false| true|
|  2|   false|   false|false|
|  4|   false|   false|false|
+---+--------+--------+-----+



In [42]:
outliers = df_outliers.select( *['id'] + [((df_outliers[c] < bounds[c][0]) | (df_outliers[c] > bounds[c][1])).alias(c + '_o') for c in cols] )


- 몸무게와 나이 피처에 각각 두 개의 아웃라이어가 있습니다. 예전에는 아웃라이어를 판단하는 공식을 따로 알아야만 했습니다. 하지만 아웃라이어를 찾아내기 위해 위의 코드만 이용하면 됩니다.


In [44]:
df_outliers = df_outliers.join(outliers, on='id') 
df_outliers.filter('weight_o').select('id', 'weight').show() 
df_outliers.filter('age_o').select('id', 'age').show()

AnalysisException: "Reference 'weight_o' is ambiguous, could be: weight_o, weight_o, weight_o, weight_o, weight_o, weight_o, weight_o, weight_o.; line 1 pos 0"

## 기술 통계
- 데이터셋에서의 관찰 값 개수, 각 컬럼의 평균과 표준 편차 또는 최댓값/최솟값 등의 기본적인 정보를 제공합니다. 
    1. 우선 데이터를 로드하고 스파크 데이터프레임으로 변환해 보겠습니다.
    1. 우선 필요한 모듈을 로드해보겠습니다. pyspark.sql.types는 IntegerType()이나 FloatType() 같은 사용 가능한 모든 데이터 타입을 로드합니다. 
    1. 그다음에는 데이터를 읽고 헤더 라인을 filter() 함수로 제거합니다. 
    1. 다음 작업은 행을 콤마로 분리하고 각각의 엘리먼트를 정수형으로 변환하는 작업입니다.



In [49]:
import pyspark.sql.types as typ

fraud = sc.textFile(base_path+'ccFraud.csv')
header = fraud.first()
header

'"custID","gender","state","cardholder","balance","numTrans","numIntlTrans","creditLine","fraudRisk"'

In [50]:
type(header)

str

In [51]:
fraud = fraud\
        .filter(lambda row: row != header)\
        .map(lambda row: [int(elem) for elem in row.split(',')])

## 자동으로 StructureField

In [52]:
# 데이터프레임 스키마 생성
fields = [
    *[typ.StructField( h[1:-1], typ.IntegerType(), True )
      for h in header.split(',')]
]
schema = typ.StructType(fields)

In [53]:
fields

[StructField(custID,IntegerType,true),
 StructField(gender,IntegerType,true),
 StructField(state,IntegerType,true),
 StructField(cardholder,IntegerType,true),
 StructField(balance,IntegerType,true),
 StructField(numTrans,IntegerType,true),
 StructField(numIntlTrans,IntegerType,true),
 StructField(creditLine,IntegerType,true),
 StructField(fraudRisk,IntegerType,true)]

In [54]:
schema

StructType(List(StructField(custID,IntegerType,true),StructField(gender,IntegerType,true),StructField(state,IntegerType,true),StructField(cardholder,IntegerType,true),StructField(balance,IntegerType,true),StructField(numTrans,IntegerType,true),StructField(numIntlTrans,IntegerType,true),StructField(creditLine,IntegerType,true),StructField(fraudRisk,IntegerType,true)))

In [55]:
# 데이터프레임 생성
fraud_df = spark.createDataFrame(fraud, schema)

In [56]:
fraud_df

DataFrame[custID: int, gender: int, state: int, cardholder: int, balance: int, numTrans: int, numIntlTrans: int, creditLine: int, fraudRisk: int]

In [57]:
fraud_df.printSchema()

root
 |-- custID: integer (nullable = true)
 |-- gender: integer (nullable = true)
 |-- state: integer (nullable = true)
 |-- cardholder: integer (nullable = true)
 |-- balance: integer (nullable = true)
 |-- numTrans: integer (nullable = true)
 |-- numIntlTrans: integer (nullable = true)
 |-- creditLine: integer (nullable = true)
 |-- fraudRisk: integer (nullable = true)



- 카테고리 컬럼을 더 자세히 이해하기 위해, groupby() 함수를 이용해 그 값들의 빈도를 셀 수 있습니다. 성별 칼럼에 대한 빈도를 세보겠습니다.

In [58]:
# 빈도수 구하기
fraud_df.groupby('gender').count().show()

+------+-------+
|gender|  count|
+------+-------+
|     1|6178231|
|     2|3821769|
+------+-------+



- 결과값에 차이가 있다면 다소 불균형한 데이터셋임을 확인할 수 있습니다. 실제 숫자 피처에 대해서는 describe() 함수를 사용할 수 있습니다.

In [60]:
# describe() 확인
numerical = ['balance', 'numTrans', 'numIntlTrans']

In [61]:
desc = fraud_df.describe(numerical)
desc.show()

+-------+-----------------+------------------+-----------------+
|summary|          balance|          numTrans|     numIntlTrans|
+-------+-----------------+------------------+-----------------+
|  count|         10000000|          10000000|         10000000|
|   mean|     4109.9199193|        28.9351871|        4.0471899|
| stddev|3996.847309737258|26.553781024523122|8.602970115863904|
|    min|                0|                 0|                0|
|    max|            41485|               100|               60|
+-------+-----------------+------------------+-----------------+



- 위의 소스를 출력하여 얻을 수 있는 적은 정보(count, mean, stddev, min, max)들을 통해서도 상당히 많은 것을 알 수 있습니다.

    - 모든 피처는 양의 방향으로 왜곡되었다. 최댓값이 평균값보다 몇배 더 크다.

    - 변동 계수 : coefficient variabion(평균과 표준 편차의 비율)가 매우 크다
        (값이 1과 가깝거나 크다. 이는 넓게 퍼진 데이터를 의미한다.)

 

- 다음 코드도 비대칭도를 확인할 수 있습니다(밸런스 피처)
    - balance 컬럼만 확인

In [62]:
fraud_df.agg({'balance': 'skewness'}).show()

+------------------+
| skewness(balance)|
+------------------+
|1.1818315552993839|
+------------------+



##  상관 계수

- 서로 다른 피처들 간의 상호 관계 표시
- 모델은 타킷 데이터에 상당히 연관돼 있는 피처만 포함

### 피어슨 상관 계수
- pyspark는 피어슨 상관계수 만을 지원 corr() 
- 오직 두 쌍의 상관 계수만 계산 가능 


In [63]:
fraud_df.corr('balance', 'numTrans')

0.0004452314017265386

In [64]:
n_numerical = len(numerical)

In [69]:
n_numerical

3

In [70]:
corr = []
# 10 분 걸리는 작업 !!!
for i in range(0, n_numerical):
    temp = [None] * i
    for j in range(i, n_numerical):
        temp.append( fraud_df.corr(numerical[i], numerical[j]) )
    corr.append(temp)

In [71]:
corr

[[1.0, 0.0004452314017265386, 0.0002713991339817875],
 [None, 1.0, -0.00028057128198165544],
 [None, None, 1.0]]

## 시각화
```py
conda install -c conda-forge bkcharts
conda install -c conda-forge holoviews
```

### .1 히스토그램
- 히스토그램은 피처의 분포를 시각화하는 가장 쉬운 방법입니다. 
- 파이스파크에서 히스토그램을 만드는 방법은 세가지입니다.
    - 첫째
        1. 데이터를 워커 노드에서 집계
        1. 워커 노드가 bin 리스트를 드라이버 노드에게 리턴하고
        1. 각 bin의 개수를 드라이버 노드가 센다
    - 둘째
    
        1. 데이터를 모두 드라이버 노드에 리턴하고
        1. 시각화 라이브러리 함수를 사용해 히스토그램을 만든다.
            - 행의 개수가 너무 많으면, 사용불가 방법. 
            - 따라서 데이터를 먼저 집계
    - 셋째
        1. 데이터를 샘플링해 
        1. 드라이버 노드에 리턴한다.
        1. 드라이버 노드는 리턴된 데이터를 이용해 데이터를 시각화 한다.


In [96]:
hist = fraud_df.select('balance').rdd.flatMap(lambda row: row).histogram(20)

In [ ]:
hist

In [ ]:
len(hist[0])

In [186]:
import matplotlib.pyplot as plt

plt.style.use('ggplot')
%matplotlib inline

AttributeError: module 'matplotlib.cbook' has no attribute '_classproperty'

In [191]:
data = {
      'bins' : hist[0][:-1]
    , 'freq' : hist[1]
}

In [192]:
fig = plt.figure(figsize = (12,9))
ax = fig.add_subplot(1,1,1)
ax.bar(data['bins'], data['freq'], width = 2000)
ax.set_title('Histogram of \'balance\'')
plt.show()
#plt.savefig('B000001.png', dpi = 360) 

NameError: name 'plt' is not defined

In [197]:
hv.Bars(data['freq'], 'freq', 'bins') + hv.Bars(data['bins'], 'freq', 'bins')

NameError: name 'hv' is not defined

- 데이터의 양이 드라이버 노드에서 처리 할 수 있을 정도로 충분히 작다면,
    - 데이터를 가져와서 matplotlib의 hist() 함수 
    - 또는 Boken의 Histogram() 함수를 사용해 
    - 히스토그램을 작성 가능 ( 그래프모양은 같다 )
    - (**앞의 방법을 사용하는 것이 일반적으로 더욱 빠르다**)
    

In [198]:
data_driver = {'obs' : fraud_df.select('balance').rdd.flatMap(lambda row:row).collect()}

In [ ]:
len(data_driver['obs'])

In [199]:
fig = plt.figure(figsize=(12,9))
ax = fig.add_subplt(1,1,1)
ax.hist(data_driver['obs'], bins=20)
ax.set_title('Histogram of \' balance \' using .hist()')
plt.show()

NameError: name 'plt' is not defined

### .2 Scatter chart

- 변수간의 상호작용을 시각화 가능

- pyspark 서버 모듈에서는 어떠한 시각화 모듈도 미제공
- 수십억개의 데이터를 동시에 시각화 하는것은 비현실적
- 데이터셋에서 0.02% 로 샘플링 하여 시각화 고려




* 피처 사이의 상호작용

 

Scatter 차트로 동시에 세 개 변수 간의 상호작용을 시각화할 수 있습니다. 임시 데이터를 다루거나 변화를 지속적으로 관찰하고 싶은 경우가 아닌 이상 세 개의 피처에 대한 시각화는 다루지 않을 것입니다. 설령 그런 경우가 있다고 하더라도 시간 데이터를 불연속적으로 구분해 2D 형태로 시각화할 것입니다. 3D 형태로 차트를 해석하면 대부분의 경우 더욱 복잡하고 헷갈리는 작업이 됩니다.

 

파이스파크는 서버 쪽에서는 어떠한 시각화 모듈도 제공하지 않기 때문에 수십억개의 데이터를 동시에 시각화하는 것은 매우 비현실 적입니다. 데이터셋은 0.02%로 샘플링(약 2,000개 데이터)하여 구현하면 훨씬 작업이 간단해질 수 있습니다.

 

계층화된 샘플링을 하지 않는 이상, 정해진 샘플링 비율로 적어도 세 개에서 다섯 개의 샘플링을 준비해야 합니다. 이렇게 함으로써 샘플링된 데이터셋이 전체를 대표할 수 있는지 체크할 수 있습니다. 


- 다음은 성별을 기준으로 0.02% Fraud 데이터셋을 샘플링하는 예제


In [201]:
data_sample = fraud_df.sampleBy('gender', {1:0.0002, 2:0.0002}).select(numerical)

In [203]:
data_sample.printSchema()

root
 |-- balance: integer (nullable = true)
 |-- numTrans: integer (nullable = true)
 |-- numIntlTrans: integer (nullable = true)



In [204]:
data_sample['balance']

Column<b'balance'>

In [206]:
data_multi = dict( [(elem, 
                     data_sample.select(elem).rdd.flatMap(lambda row: row).collect()
                    ) for elem in numerical] )

In [207]:
# sctr = chrt.Scatter( data_multi, x='balance', y='numTrans')
# chrt.show(sctr)
hv.Scatter( data_multi, x='balance', y='numTrans')

NameError: name 'hv' is not defined

1. 지원자님을 하나의 자동차(브랜드 무관)에 비유하여 설명 해주시고 이번 Global Internship(H-Experience)를 통해 얻고 싶은 목표를 서술 해주세요. (1000자)

2. 해당 직무를 선택한 이유와 지원자님께서 본 분야에 적합하다고 판단할 수 있는 근거를 서술 해주십시오. (1000자)

3. 본인의 장점 혹은 직무 역량을 나타낼 수 있는 주요 사례를 선정하여, 해당 경험을 통해 체득한 장점 혹은 직무 역량을 간략하게 서술 해주십시오. 
※ 석/박사 과정이신 경우, 연구경력 및 세부전공 중심으로 서술 해주십시오. ※ 본인을 표현할 수 있는 어떤 이벤트도 좋습니다. 자유롭게 서술 해주십시오. (1000자)